# Import and Reshape the dataframe

In [1]:
import math
from statistics import mean 
import itertools
from itertools import chain 

In [2]:
import pandas as pd
import numpy as np
df = pd.read_excel('epi_study.xlsx')
a = len(df.index)
print(a, "rows" )

44573 rows


In [3]:
df.head(20)

,Person,Age,DoB,Injury,Sex,Date of Attendance
0,1,31.817224,11/04/1989,NaN,Male,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Mechanism of Injury,NaN,NaN,O2 - Overtraining,NaN,22/08/2019
3,Stage,NaN,NaN,Sub-Acute 1-2 Months,NaN,22/08/2019
4,Body Location,NaN,NaN,Other,NaN,22/08/2019
5,2,49.025352,25/01/1972,NaN,Male,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN
7,Stage,NaN,NaN,Chronic 10Years+,NaN,14/11/2016
8,Mechanism of Injury,NaN,NaN,T1 - Twist,NaN,14/11/2016
9,Body Location,NaN,NaN,Hip,NaN,14/11/2016


Ok so the data structure is less than desirable. We have to try to find a way to make it more readable. From scanning the outputs and the excel file, every time a number appears in the 'Person' column, that indicates a new person. Each person could have multiple visits, so for example rows 10 - 19 above are person 3, while 5-9 are person 2. Makes it tricky. First thing is to remove rows that have only NaN entries.

In [4]:
df = df.dropna(how='all')
b = len(df.index)
print("Removed", a-b ,"NaN rows" )

Removed 9765 NaN rows


In [5]:
df = df.reset_index(drop=True)
df.head(20)

,Person,Age,DoB,Injury,Sex,Date of Attendance
0,1,31.817224,11/04/1989,NaN,Male,NaN
1,Mechanism of Injury,NaN,NaN,O2 - Overtraining,NaN,22/08/2019
2,Stage,NaN,NaN,Sub-Acute 1-2 Months,NaN,22/08/2019
3,Body Location,NaN,NaN,Other,NaN,22/08/2019
4,2,49.025352,25/01/1972,NaN,Male,NaN
5,Stage,NaN,NaN,Chronic 10Years+,NaN,14/11/2016
6,Mechanism of Injury,NaN,NaN,T1 - Twist,NaN,14/11/2016
7,Body Location,NaN,NaN,Hip,NaN,14/11/2016
8,3,28.902172,10/03/1992,NaN,Male,NaN
9,Mechanism of Injury,NaN,NaN,G5 - Not Known,NaN,05/10/2017


We can create a new dataframe containing more appropriate columns, then write a function to fill it out properly.

In [6]:
column_names = ["Person", "Age", "Mechanism","Stage","Location","Sex","Date of Attendance"]
# The Integer colum is explained in the next section.
df_epi = pd.DataFrame(columns = column_names)

I think the key to this is that there seems to be a fixed number of rows for each visit - looks like 3. So the number of rows taking up a single visit is 3, 6 for two visits etc. There is then the one additional row per person for their index, age and DOB. Lets see if we can validate this. Lets create a new column in the original dataframe called Integer, which is a boolen TRUE/FALSE if the entry in the Person column is an integer.

We need a way to count the number of visits each person had. Then we can place the correct number of rows per person in the new dataframe.

In [7]:
visits_per_person = [] # Define the first one for ease of looping
previous_row_num = 0 # Keep track of the previous row number
for i in range(1,len(df.index)):
    if type(df.iloc[i]['Person']) == int:
        # Get the row number of the current integer
        visits = (i - previous_row_num -1)/3
        previous_row_num = i
        visits_per_person.append(visits)

In [8]:
visits_per_person[0:10]

[1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0]

In [9]:
df.tail(20)

,Person,Age,DoB,Injury,Sex,Date of Attendance
34788,Stage,NaN,NaN,Acute 7-14 Days,NaN,07/01/2020
34789,Mechanism of Injury,NaN,NaN,T5 - Running,NaN,07/01/2020
34790,NaN,23.855547,28/03/1997,NaN,Female,NaN
34791,Body Location,NaN,NaN,Knee,NaN,08/11/2016
34792,Stage,NaN,NaN,Acute 1-7 Days,NaN,08/11/2016
34793,Mechanism of Injury,NaN,NaN,T5 - Running,NaN,08/11/2016
34794,Mechanism of Injury,NaN,NaN,T5 - Running,NaN,04/05/2017
34795,Stage,NaN,NaN,Acute 1-7 Days,NaN,04/05/2017
34796,Body Location,NaN,NaN,Hamstring,NaN,04/05/2017
34797,NaN,22.384668,16/09/1998,NaN,Male,NaN


Ok I don't think the person number is reliable. Instead we should look at Age. That appears to be a better column.

In [10]:
visits_per_person = [1] # Define the first one for ease of looping
previous_row_num = 0 # Keep track of the previous row number
for i in range(0,len(df.index)):
    if not math.isnan(df.iloc[i]['Age']):
        # Get the row number of the current integer
        visits = (i - previous_row_num -1)/3
        previous_row_num = i
        visits_per_person.append(visits)
visits_per_person.append(2.0) # Adding in the last person manually
visits_per_person.pop(1) # Remove the first entry of this loop because its dodgy
visits_per_person.pop(0)

1

In [11]:
visits_per_person[0:10]

[1.0, 1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 1.0]

In [12]:
visits_per_person[(len(visits_per_person)-10):len(visits_per_person)]

[3.0, 1.0, 1.0, 3.0, 3.0, 1.0, 1.0, 2.0, 1.0, 2.0]

This looks right. Summarise this info:

In [13]:
print(len(visits_per_person),"people were seen in the period, and there was a total of",int(sum(visits_per_person)),"appointments. The average number of appointments  per person was",round(mean(visits_per_person),2),". The person with the highest number of appointments had",int(max(visits_per_person)),", and the least had",min(visits_per_person),".")

5513 people were seen in the period, and there was a total of 9765 appointments. The average number of appointments  per person was 1.77 . The person with the highest number of appointments had 11 , and the least had 1.0 .


In [14]:
df.head(10)

,Person,Age,DoB,Injury,Sex,Date of Attendance
0,1,31.817224,11/04/1989,NaN,Male,NaN
1,Mechanism of Injury,NaN,NaN,O2 - Overtraining,NaN,22/08/2019
2,Stage,NaN,NaN,Sub-Acute 1-2 Months,NaN,22/08/2019
3,Body Location,NaN,NaN,Other,NaN,22/08/2019
4,2,49.025352,25/01/1972,NaN,Male,NaN
5,Stage,NaN,NaN,Chronic 10Years+,NaN,14/11/2016
6,Mechanism of Injury,NaN,NaN,T1 - Twist,NaN,14/11/2016
7,Body Location,NaN,NaN,Hip,NaN,14/11/2016
8,3,28.902172,10/03/1992,NaN,Male,NaN
9,Mechanism of Injury,NaN,NaN,G5 - Not Known,NaN,05/10/2017


Lets try to fill out the new dataframe's column 'Person' with the correct number of appearances. We can start by defining a new vector and repeating the person's number for each visit. Some day I'll stop relying on for loops...

In [15]:
# Convert the vector visits_per_person to an int using a list comprehension
x = [int(x) for x in visits_per_person]
#x.pop(0)
person_list = [] # Empty list which will be our new column
for i in range(1,(len(x)+1)):
    person_list.append(list(itertools.repeat(i,int(visits_per_person[i-1]))))
person_list = list(chain.from_iterable(person_list))
#person_list.pop(0)

In [16]:
person_list[0:10]

[1, 2, 3, 3, 3, 4, 5, 6, 7, 8]

In [17]:
person_list[-10:]

[5508, 5508, 5508, 5509, 5510, 5511, 5511, 5512, 5513, 5513]

Ok it looks right, so lets put it into the new dataframe:

In [18]:
df_epi["Person"] = person_list
df_epi.head(10)

,Person,Age,Mechanism,Stage,Location,Sex,Date of Attendance
0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,3,NaN,NaN,NaN,NaN,NaN,NaN
5,4,NaN,NaN,NaN,NaN,NaN,NaN
6,5,NaN,NaN,NaN,NaN,NaN,NaN
7,6,NaN,NaN,NaN,NaN,NaN,NaN
8,7,NaN,NaN,NaN,NaN,NaN,NaN
9,8,NaN,NaN,NaN,NaN,NaN,NaN


Now lets work on the Age. Lets first extract a list of all ages. We'll have to remove NaN values.

In [19]:
ages = df["Age"]
ages = [x for x in ages if str(x) != 'nan']
len(ages)

5513

Perfect, 5513 ages. Checks out with the total above, so none missing. Lets make the ages more reasonable - to one decimal place. Then we can insert into the dataframe in the same way as above.

In [20]:
round_ages = [round(age, 1) for age in ages]
age_list = [] # Empty list which will be our new column
for i in range(1,(len(round_ages)+1)):
    age_list.append(list(itertools.repeat(round_ages[i-1],int(visits_per_person[i-1]))))
age_list = list(chain.from_iterable(age_list))

In [21]:
age_list[0:10]

[31.8, 49.0, 28.9, 28.9, 28.9, 25.8, 32.6, 39.5, 54.6, 61.6]

In [22]:
age_list[-10:]

[30.0, 30.0, 30.0, 52.1, 25.1, 23.9, 23.9, 22.4, 28.5, 28.5]

In [23]:
df_epi["Age"] = age_list
df_epi.head(10)

,Person,Age,Mechanism,Stage,Location,Sex,Date of Attendance
0,1,31.8,NaN,NaN,NaN,NaN,NaN
1,2,49.0,NaN,NaN,NaN,NaN,NaN
2,3,28.9,NaN,NaN,NaN,NaN,NaN
3,3,28.9,NaN,NaN,NaN,NaN,NaN
4,3,28.9,NaN,NaN,NaN,NaN,NaN
5,4,25.8,NaN,NaN,NaN,NaN,NaN
6,5,32.6,NaN,NaN,NaN,NaN,NaN
7,6,39.5,NaN,NaN,NaN,NaN,NaN
8,7,54.6,NaN,NaN,NaN,NaN,NaN
9,8,61.6,NaN,NaN,NaN,NaN,NaN


In [24]:
df_epi.tail(10)

,Person,Age,Mechanism,Stage,Location,Sex,Date of Attendance
9755,5508,30.0,NaN,NaN,NaN,NaN,NaN
9756,5508,30.0,NaN,NaN,NaN,NaN,NaN
9757,5508,30.0,NaN,NaN,NaN,NaN,NaN
9758,5509,52.1,NaN,NaN,NaN,NaN,NaN
9759,5510,25.1,NaN,NaN,NaN,NaN,NaN
9760,5511,23.9,NaN,NaN,NaN,NaN,NaN
9761,5511,23.9,NaN,NaN,NaN,NaN,NaN
9762,5512,22.4,NaN,NaN,NaN,NaN,NaN
9763,5513,28.5,NaN,NaN,NaN,NaN,NaN
9764,5513,28.5,NaN,NaN,NaN,NaN,NaN


Ok, so far so good. Lets do the Sex now, same mechanism.

In [25]:
sex = df["Sex"]
sex = [x for x in sex if str(x) != 'nan']
len(sex)

5513

In [26]:
#round_ages = [round(age, 1) for age in ages]
sex_list = [] # Empty list which will be our new column
for i in range(1,(len(sex)+1)):
    sex_list.append(list(itertools.repeat(sex[i-1],int(visits_per_person[i-1]))))
sex_list = list(chain.from_iterable(sex_list))

In [27]:
sex_list[0:10]

['Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male',
 'Male']

In [28]:
df_epi["Sex"] = sex_list
df_epi.head(10)

,Person,Age,Mechanism,Stage,Location,Sex,Date of Attendance
0,1,31.8,NaN,NaN,NaN,Male,NaN
1,2,49.0,NaN,NaN,NaN,Male,NaN
2,3,28.9,NaN,NaN,NaN,Male,NaN
3,3,28.9,NaN,NaN,NaN,Male,NaN
4,3,28.9,NaN,NaN,NaN,Male,NaN
5,4,25.8,NaN,NaN,NaN,Male,NaN
6,5,32.6,NaN,NaN,NaN,Male,NaN
7,6,39.5,NaN,NaN,NaN,Male,NaN
8,7,54.6,NaN,NaN,NaN,Male,NaN
9,8,61.6,NaN,NaN,NaN,Male,NaN


That's the easy part done. Now we need to figure out a way to take each visit's Mechanism, Stage, Location and Date of Visit. Lets remove all the rows that contain the information we already have (ie person number, age, sex). We can remove all rows that have an integer in the person column.

In [29]:
df_filtered = df
df_filtered.dropna(subset = ["Injury"], inplace=True)
df_filtered.head(10)

,Person,Age,DoB,Injury,Sex,Date of Attendance
1,Mechanism of Injury,NaN,NaN,O2 - Overtraining,NaN,22/08/2019
2,Stage,NaN,NaN,Sub-Acute 1-2 Months,NaN,22/08/2019
3,Body Location,NaN,NaN,Other,NaN,22/08/2019
5,Stage,NaN,NaN,Chronic 10Years+,NaN,14/11/2016
6,Mechanism of Injury,NaN,NaN,T1 - Twist,NaN,14/11/2016
7,Body Location,NaN,NaN,Hip,NaN,14/11/2016
9,Mechanism of Injury,NaN,NaN,G5 - Not Known,NaN,05/10/2017
10,Body Location,NaN,NaN,Calf,NaN,05/10/2017
11,Stage,NaN,NaN,Acute 1-7 Days,NaN,05/10/2017
12,Stage,NaN,NaN,Acute 1-7 Days,NaN,09/10/2017


Put the person's number in as a new column, to help us see what to do!

In [30]:
df_filtered["Person_Number"] = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in person_list))
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.head(20)

,Person,Age,DoB,Injury,Sex,Date of Attendance,Person_Number
0,Mechanism of Injury,NaN,NaN,O2 - Overtraining,NaN,22/08/2019,1
1,Stage,NaN,NaN,Sub-Acute 1-2 Months,NaN,22/08/2019,1
2,Body Location,NaN,NaN,Other,NaN,22/08/2019,1
3,Stage,NaN,NaN,Chronic 10Years+,NaN,14/11/2016,2
4,Mechanism of Injury,NaN,NaN,T1 - Twist,NaN,14/11/2016,2
5,Body Location,NaN,NaN,Hip,NaN,14/11/2016,2
6,Mechanism of Injury,NaN,NaN,G5 - Not Known,NaN,05/10/2017,3
7,Body Location,NaN,NaN,Calf,NaN,05/10/2017,3
8,Stage,NaN,NaN,Acute 1-7 Days,NaN,05/10/2017,3
9,Stage,NaN,NaN,Acute 1-7 Days,NaN,09/10/2017,3


In [31]:
test = df_epi

The next series of commands are to put the miscellanous variables from the Person column into their respective columns in the new dataframe.

In [33]:
for i in set(person_list):
    subset_i = df_filtered[df_filtered.Person_Number == i]
    # Loop through the rows in each subset
    z = []
    for index, row in subset_i.iterrows():
        if row['Person'] == "Stage":
            z.append(row['Injury'])
        target_row = test[test['Person']==i].index.values[0]
    test['Stage'][target_row:(target_row+person_list.count(i))] = z   # test is a copy of the df_epi dataframe, change it once it works 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [34]:
for i in set(person_list):
    subset_i = df_filtered[df_filtered.Person_Number == i]
    # Loop through the rows in each subset
    z = []
    for index, row in subset_i.iterrows():
        if row['Person'] == "Body Location":
            z.append(row['Injury'])
        target_row = test[test['Person']==i].index.values[0]
    test['Location'][target_row:(target_row+person_list.count(i))] = z   # test is a copy of the df_epi dataframe, change it once it works 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [35]:
for i in set(person_list):
    subset_i = df_filtered[df_filtered.Person_Number == i]
    # Loop through the rows in each subset
    z = []
    for index, row in subset_i.iterrows():
        if row['Person'] == "Mechanism of Injury":
            z.append(row['Injury'])
        target_row = test[test['Person']==i].index.values[0]
    test['Mechanism'][target_row:(target_row+person_list.count(i))] = z   # test is a copy of the df_epi dataframe, change it once it works 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


The one for Date of Attendance is a bit different:

In [36]:
for i in set(person_list):
    subset_i = df_filtered[df_filtered.Person_Number == i]
    # Loop through the rows in each subset
    x = []
    z = []
    for index, row in subset_i.iterrows():
        x.append(row["Date  of Attendance"])
    z = list(x[j] for j in range(0,person_list.count(i)*3,3))
    target_row = test[test['Person']==i].index.values[0]
    test['Date of Attendance'][target_row:(target_row+person_list.count(i))] = z   # test is a copy of the df_epi dataframe, change it once it works 


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [38]:
df_epi = test # Yeah, I know I know

In [39]:
df_epi.head(20)

,Person,Age,Mechanism,Stage,Location,Sex,Date of Attendance
0,1,31.8,O2 - Overtraining,Sub-Acute 1-2 Months,Other,Male,22/08/2019
1,2,49.0,T1 - Twist,Chronic 10Years+,Hip,Male,14/11/2016
2,3,28.9,G5 - Not Known,Acute 1-7 Days,Calf,Male,05/10/2017
3,3,28.9,T5 - Running,Acute 1-7 Days,Ankle,Male,09/10/2017
4,3,28.9,T7 - Over-Stretch,Sub-Acute 21-28 Days,Shoulder,Male,07/05/2020
5,4,25.8,T16 - Tackling,Sub-Acute 1-2 Months,Fingers & Thumb,Male,22/11/2017
6,5,32.6,T1 - Twist,Sub-Acute 14-21 Days,Ankle,Male,10/08/2017
7,6,39.5,G5 - Not Known,Chronic 6-12 Months,Cervical,Male,11/06/2019
8,7,54.6,O1 - Repetitive Strain Injury,Chronic 1-5 Years,Lumbar,Male,09/01/2018
9,8,61.6,G5 - Not Known,Acute 7-14 Days,Knee,Male,08/11/2016


Ready for some analysis now!

## Exploratory Data Analysis

Coming soon!